### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

In [2]:
# Files to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [3]:
# Read School and Student Data File and store into Pandas Data Frames
school_df = pd.read_csv(school_data_to_load)
student_df = pd.read_csv(student_data_to_load)


In [4]:
# Combine the data into a single dataset
school_data_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
school_data_df.head()
#school_data_df.dtypes

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [5]:
#Calculate the total number of schools
school_count = school_data_df["school_name"].nunique()
#print(school_count)

In [6]:
#Calculate the total number of students
student_count = school_data_df["student_name"].nunique()
#print(student_count)

In [7]:
#Calculate the total budget
#total_budget = school_data_df["budget"].nunique()-ran to verify each school has a unique budget
total_budget = school_data_df["budget"].unique().sum()
#print(total_budget)

In [8]:
#Calculate the average math score
avg_math = round(school_data_df["math_score"].mean(),2)
#print(avg_math)

In [9]:
#Calculate the average reading score
avg_read = round(school_data_df["reading_score"].mean(),2)
#print(avg_read)

In [10]:
#Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_avg_score = round((avg_math+avg_read)/2,2)
#print(overall_avg_score)

In [11]:
#Calculate the percentage of students with a passing math score (70 or greater)
pass_math = round(school_data_df["math_score"].loc[school_data_df["math_score"]>70].count()/student_count*100,2)
#print(pass_math)

In [12]:
#Calculate the percentage of students with a passing reading score (70 or greater)
pass_read = round(school_data_df["reading_score"].loc[school_data_df["reading_score"]>70].count()/student_count*100,2)
#print(pass_read)

In [13]:
#Create a dataframe to hold the above results
district_data = {"Schools":school_count,
                 "Students":student_count,
                 "Budget":total_budget,
                 "Average Math":avg_math,
                 "Average Reading":avg_read,
                 "Overall Average Score":overall_avg_score,
                 "% Passing Math":pass_math,
                 "% Passing Reading":pass_read}
district_df = pd.DataFrame(data=district_data, index=["District 1"])
district_df.head()

,Schools,Students,Budget,Average Math,Average Reading,Overall Average Score,% Passing Math,% Passing Reading
District 1,15,32715,24649428,78.99,81.88,80.44,86.68,99.34


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [14]:
bins = [0,69.9,100]
group_names = [0,1]
school_data_df["pass_fail_math"] = pd.cut(school_data_df["math_score"], bins, labels=group_names, include_lowest=True)
school_data_df["pass_fail_reading"] = pd.cut(school_data_df["reading_score"], bins, labels=group_names, include_lowest=True)
#school_data_df.head()

In [15]:
school_data_df["pass_fail_math"] = school_data_df["pass_fail_math"].astype("int64")
school_data_df["pass_fail_reading"] = school_data_df["pass_fail_reading"].astype("int64")
#school_data_df.dtypes

In [16]:
#group by first for type and budget
school_summary_agg_df = school_data_df.groupby("school_name").agg({"type": ["first"],
                                                                   "student_name": ["count"],
                                                                   "budget":["first"],
                                                                   "math_score": ["mean"],
                                                                   "reading_score": ["mean"],
                                                                   "pass_fail_math":["sum"],
                                                                   "pass_fail_reading":["sum"]})
#school_summary_agg_df

In [17]:
# rename columns
school_summary_agg_df.columns = ['school_type','student_count','school_budget','avg_math','avg_reading','passed_math','passed_reading']

# reset index to get grouped columns back
school_summary_agg_df = school_summary_agg_df.reset_index()

#school_summary_agg_df.head()

In [18]:
# add calculated columns
school_summary_agg_df["per_student_budget"] = school_summary_agg_df["school_budget"]/school_summary_agg_df["student_count"]
school_summary_agg_df["%pass_math"] = round(school_summary_agg_df["passed_math"]/school_summary_agg_df["student_count"]*100,2)
school_summary_agg_df["%pass_reading"] = round(school_summary_agg_df["passed_reading"]/school_summary_agg_df["student_count"]*100,2)
school_summary_agg_df["total_pass%"] = (school_summary_agg_df["%pass_math"]+school_summary_agg_df["%pass_reading"])/2
#school_summary_agg_df.head()

In [19]:
#drop extra columns and re-arrange
by_school_df = school_summary_agg_df.drop(school_summary_agg_df.columns[[6,7]], axis=1)


In [20]:
#reorder all columns
by_school_df = school_summary_agg_df[["school_name","school_type","student_count","school_budget","per_student_budget","avg_reading","avg_math","%pass_reading","%pass_math","total_pass%"]]
by_school_df

,school_name,school_type,student_count,school_budget,per_student_budget,avg_reading,avg_math,%pass_reading,%pass_math,total_pass%
0,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,81.93,66.68,74.305
1,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.04,94.13,95.585
2,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.74,65.99,73.365
3,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,79.30,68.31,73.805
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.14,93.39,95.265
5,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,80.86,66.75,73.805
6,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,96.25,92.51,94.380
7,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.32,65.68,73.500
8,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,81.22,66.06,73.640
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.95,94.59,95.270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [21]:
#using previously grouped data retrive the 5 highest pass rates and put them in order
top_performing_df = by_school_df.nlargest(5,"total_pass%")
top_performing_df.sort_values(by=["total_pass%"])
top_performing_df

,school_name,school_type,student_count,school_budget,per_student_budget,avg_reading,avg_math,%pass_reading,%pass_math,total_pass%
1,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.04,94.13,95.585
12,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,97.31,93.27,95.290
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.95,94.59,95.270
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.14,93.39,95.265
13,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,96.54,93.87,95.205


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [22]:
#using previously grouped data retrive the 5 lowest pass rates and put them in order
top_performing_df = by_school_df.nsmallest(5,"total_pass%")
top_performing_df.sort_values(by=["total_pass%"], ascending=False)
top_performing_df

,school_name,school_type,student_count,school_budget,per_student_budget,avg_reading,avg_math,%pass_reading,%pass_math,total_pass%
10,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,80.22,66.37,73.295
2,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.74,65.99,73.365
7,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.32,65.68,73.500
8,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,81.22,66.06,73.640
3,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,79.30,68.31,73.805


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [23]:
#Create a pandas series for each grade- grouped by school
nineth_grade_df = school_data_df.loc[school_data_df["grade"] == "9th",["school_name","math_score",]].groupby("school_name")["math_score"].mean()
tenth_grade_df = school_data_df.loc[school_data_df["grade"] == "10th",["school_name","math_score",]].groupby("school_name")["math_score"].mean()
eleventh_grade_df = school_data_df.loc[school_data_df["grade"] == "11th",["school_name","math_score",]].groupby("school_name")["math_score"].mean()
twelth_grade_df = school_data_df.loc[school_data_df["grade"] == "12th",["school_name","math_score",]].groupby("school_name")["math_score"].mean()
#nineth_grade_df

In [24]:
#Combine the series into a dataframe using suffixes to label
math_by_grade1_df = pd.merge(nineth_grade_df, tenth_grade_df, on="school_name", how="left", suffixes=("_9th", "_10th"))
math_by_grade2_df = pd.merge(eleventh_grade_df, twelth_grade_df, on="school_name", how="left", suffixes=("_11th", "_12th"))
math_by_grade_df = pd.merge(math_by_grade1_df, math_by_grade2_df, on="school_name", how="left")

math_by_grade_df

,math_score_9th,math_score_10th,math_score_11th,math_score_12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [25]:
#Create a pandas series for each grade- grouped by school
nineth_r_grade_df = school_data_df.loc[school_data_df["grade"] == "9th",["school_name","reading_score",]].groupby("school_name")["reading_score"].mean()
tenth_r_grade_df = school_data_df.loc[school_data_df["grade"] == "10th",["school_name","reading_score",]].groupby("school_name")["reading_score"].mean()
eleventh_r_grade_df = school_data_df.loc[school_data_df["grade"] == "11th",["school_name","reading_score",]].groupby("school_name")["reading_score"].mean()
twelth_r_grade_df = school_data_df.loc[school_data_df["grade"] == "12th",["school_name","reading_score",]].groupby("school_name")["reading_score"].mean()
#nineth_r_grade_df

In [26]:
#Combine the series into a dataframe using suffixes to label
reading_by_grade1_df = pd.merge(nineth_r_grade_df, tenth_r_grade_df, on="school_name", how="left", suffixes=("_9th", "_10th"))
reading_by_grade2_df = pd.merge(eleventh_r_grade_df, twelth_r_grade_df, on="school_name", how="left", suffixes=("_11th", "_12th"))
reading_by_grade_df = pd.merge(reading_by_grade1_df, reading_by_grade2_df, on="school_name", how="left")

reading_by_grade_df

,reading_score_9th,reading_score_10th,reading_score_11th,reading_score_12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [27]:
#school_summary_agg_df["per_student_budget"].describe()

In [28]:
# bins and names
spending_bins = [0, 585, 615, 645, 675]
group_name = ["<$585", "$585-615", "$615-645", "$645-675"]

In [29]:
#use the groupby/agg functions to get the requested averages by spending categories
scores_df = school_summary_agg_df.drop(school_summary_agg_df.columns[[3,6]], axis=1)
scores_df["scores_by_spending"] = pd.cut(scores_df["per_student_budget"], spending_bins, labels=group_name, include_lowest=True)
scores_df = scores_df.groupby("scores_by_spending").agg({"avg_math": ["mean"],
                                                         "avg_reading": ["mean"],
                                                         "%pass_math": ["mean"],
                                                         "%pass_reading": ["mean"],
                                                         "total_pass%": ["mean"]})
scores_df

,avg_math,avg_reading,%pass_math,%pass_reading,total_pass%
,mean,mean,mean,mean,mean
scores_by_spending,,,,,
<$585,83.455399,83.933814,93.460000,96.610000,95.035000
$585-615,83.599686,83.885211,94.230000,95.900000,95.065000
$615-645,79.079225,81.891436,75.668333,86.106667,80.887500
$645-675,76.997210,81.027843,66.163333,81.133333,73.648333


## Scores by School Size

* Perform the same operations as above, based on school size.

In [30]:
#school_summary_agg_df["student_count"].describe()

In [31]:
# bins and names
size_bins = [0, 500, 1000, 2000, 3500, 5000]
group_names = ["Extra Small (<500)","Small (500-1000)", "Medium (1000-2000)", "Large (2000-3500)","Extra Large (>3500)"]

In [32]:
#use the groupby/agg functions to get the requested averages by size categories
size_df = school_summary_agg_df.drop(school_summary_agg_df.columns[[3,6]], axis=1)
size_df["scores_by_size"] = pd.cut(size_df["student_count"], size_bins, labels=group_names, include_lowest=True)
size_df = size_df.groupby("scores_by_size").agg({"avg_math": ["mean"],
                                                         "avg_reading": ["mean"],
                                                         "%pass_math": ["mean"],
                                                         "%pass_reading": ["mean"],
                                                         "total_pass%": ["mean"]})
size_df

,avg_math,avg_reading,%pass_math,%pass_reading,total_pass%
,mean,mean,mean,mean,mean
scores_by_size,,,,,
Extra Small (<500),83.803279,83.814988,92.5100,96.2500,94.38000
Small (500-1000),83.839917,84.044699,94.5900,95.9500,95.27000
Medium (1000-2000),83.374684,83.864438,93.5980,96.7900,95.19400
Large (2000-3500),78.429493,81.769122,73.4625,84.4750,78.96875
Extra Large (>3500),77.063340,80.919864,66.4650,81.0575,73.76125


## Scores by School Type

* Perform the same operations as above, based on school type.

In [33]:
#use the groupby/agg functions to get the requested averages by type categories
type_df = school_summary_agg_df.drop(school_summary_agg_df.columns[[3,6]], axis=1)
type_df = type_df.groupby("school_type").agg({"avg_math": ["mean"],
                                       "avg_reading": ["mean"],
                                       "%pass_math": ["mean"],
                                       "%pass_reading": ["mean"],
                                       "total_pass%": ["mean"]})
type_df

,avg_math,avg_reading,%pass_math,%pass_reading,total_pass%
,mean,mean,mean,mean,mean
school_type,,,,,
Charter,83.473852,83.896421,93.620000,96.586250,95.103125
District,76.956733,80.966636,66.548571,80.798571,73.673571
